In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv
/kaggle/input/netflix-stock-data/NFLX.csv


In [12]:
netflix_movies = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
netflix_stocks = pd.read_csv('../input/netflix-stock-data/NFLX.csv')

In [13]:
netflix_movies = netflix_movies[netflix_movies['type']=='Movie'].dropna()
netflix_movies.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
6,s7,Movie,122,Yasir Al Yasiri,"Amina Khalil, Ahmed Dawood, Tarek Lotfy, Ahmed...",Egypt,"June 1, 2020",2019,TV-MA,95 min,"Horror Movies, International Movies","After an awful accident, a couple admitted to ..."


In [14]:
netflix_stocks.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-05-23,1.156429,1.242857,1.145714,1.196429,1.196429,104790000
1,2002-05-24,1.214286,1.225000,1.197143,1.210000,1.210000,11104800
2,2002-05-28,1.213571,1.232143,1.157143,1.157143,1.157143,6609400
3,2002-05-29,1.164286,1.164286,1.085714,1.103571,1.103571,6757800
4,2002-05-30,1.107857,1.107857,1.071429,1.071429,1.071429,10154200


Now, that we have our DataFrames ready; we can continue to doing some investigative work. Let's check the descriptive statistics for both stocks and a couple different segmentations for movies with pd.DataFrame.describe().

In [15]:
netflix_movies.date_added = pd.to_datetime(netflix_movies.date_added)
movies_over_time = netflix_movies.copy().groupby('release_year')['title'].count().reset_index()
movies_over_time = movies_over_time.rename(columns={0:'count'})

median_year = int(netflix_movies ['release_year'].median()) #2016
mode_year = int(netflix_movies ['release_year'].mode())
print("this is the median:", median_year, ", this is the mode:", mode_year)
top_movie_year = movies_over_time[movies_over_time['title']==movies_over_time['title'].max()]
movies_over_time.describe()

this is the median: 2016 , this is the mode: 2017


,release_year,title
count,71.000000,71.000000
mean,1985.295775,65.816901
std,21.827502,140.246040
min,1942.000000,1.000000
25%,1968.500000,3.000000
50%,1986.000000,9.000000
75%,2003.500000,39.000000
max,2021.000000,627.000000


In [16]:
netflix_stocks['Date'] = pd.to_datetime(netflix_stocks['Date'])

netflix_stocks.describe()

,Open,High,Low,Close,Adj Close,Volume
count,4581.000000,4581.000000,4581.000000,4581.000000,4581.000000,4.581000e+03
mean,78.031471,79.285690,76.736079,78.072056,78.072056,1.760516e+07
std,118.372084,120.276839,116.376768,118.435303,118.435303,1.971130e+07
min,0.377857,0.410714,0.346429,0.372857,0.372857,2.856000e+05
25%,3.778571,3.857143,3.685714,3.784286,3.784286,6.908900e+06
50%,15.392858,15.671429,14.970000,15.324286,15.324286,1.155350e+07
75%,97.830002,99.529999,96.760002,97.889999,97.889999,2.113440e+07
max,567.979980,575.369995,520.960022,548.729980,548.729980,3.234140e+08


We get to see our maxes and mins & highs and lows. We have a pretty good idea of the distribution for the movies and the average closing dollar amount. Could Netflix releases over the years correlated with the stock price? Can we tell if our answer is statistically significant?

If you think about releases, there are four elements of releases that might catch a movie lover's attention. You have:
1. Main film location:  language matters but relativity to the elements in the movie and being familiar with the references is quentissential in most instances. 
2. Director: we all have our favorite director. It's the movie that was most memorable or the one that made you laugh the most. You loved the cinamatic touches they added. You strap in for a good movie when you hear what's on for movie night.
3. Actor: Leonardo DiCaprio gets my buy in on any movie. We all have that person of a few! Is their a primary actor driving the market?
4. Genre: I love a good horror movie, don't you? Not your forte? Let's see if the genres of the movie releases and the stock price have a relationship.

These ideas are all speculative, but I invite my readers to stay the course. The insights and graphs are at the bottom. Continue reading if you'd like.

In [17]:
# Assuming the first movie listed is where this movie was filmed the most, we will replace the lists of movies
# with the first country and use that to find the top 10 countries that have movies are currently on Netflix 

movies_with_countries = netflix_movies.copy()
new=movies_with_countries['country'].str.split(',', n=5, expand=True)
countries = [('1st',0), ('2nd',1),('3rd',2),('4th',3),('5th',4),('6th',5)]
for i in countries:
    movies_with_countries[i[0]+" country"] = new[i[1]]
new_mwc = movies_with_countries.drop(columns=['show_id','type','director','cast'])
distribution_df = new_mwc[['1st country',
                                         'title']].groupby(
    '1st country'
).count().reset_index()
distribution_df.head()

,1st country,title
0,Argentina,45
1,Australia,49
2,Austria,5
3,Bangladesh,2
4,Belgium,5


We could say Hollywood knows how to make a good movie? Or should we say that one director is taking center stage and we just can't get enough?

In [18]:
directors = movies_with_countries.copy()
directors = directors[directors['country']=='United States'][['director','title','rating','listed_in']]
directors.head()


new_dir = directors['director'].str.split(',', n=1, expand=True)
split = [('1st',0), ('2nd',1)]

# does genre matter?
for i in split:
    directors[i[0]+" Named Director"] = new_dir[i[1]]
top_directors = directors[['1st Named Director','title','rating','listed_in']].groupby(
    '1st Named Director'
)['title'].count().reset_index()


top_directors[top_directors['title']==top_directors.title.max()]

,1st Named Director,title
495,Jay Karas,15


Let's be honest, Jay Karas is taking the market by storm with his work in comedy, so this one isn't too much of a surprise! Let's look for other trends that are in the data. Later, we will investigate if there is a correlation between the movies released and the stock price. We can't say conclusively that there is a direct cause and effect relationship; it's interested to find patterns in data, just for fun!

Now, we have to ask, *does the year a movie was released matter?* I can imagine that there have been times that the film industry took a hit and not too many movies reached production. We can only speculate about the greater implications. It could just mean that the most popular year had the best movies. We decide what this metric becomes in the end! 

In [19]:
# does genre matter?
genre = []
for i in genre:
    directors[i[0]+" genre"] = genre[i[1]]
top_directors = directors[['director','title','rating','listed_in']].groupby(
    'director'
)['title'].count().reset_index()

In [24]:
import altair as alt

titles_per_country = alt.Chart(distribution_df,
                              height=300,
                              width=500,
                              title='Number of Titles per Country').transform_window(
    rank='rank(title)',
    sort=[alt.SortField('title', order='descending')]
).transform_filter(
    alt.datum.rank <=10
).mark_bar(color='red').encode(
    y=alt.Y('title:Q',
           axis=alt.Axis(title='Number of Movies',
                         tickCount=4,
                         labelPadding=10,
                        titlePadding=10)),
    x=alt.X('1st country:N',
           sort=alt.EncodingSortField('title',order='descending'),
           title=None,
           axis=alt.Axis(
                        labelAngle=-45,
               labelFontSize=15
           )),
    color=alt.condition(
        alt.datum.rank == 1,
        alt.value('red'),
        alt.value('black')
    ),
    tooltip=['1st country', 'title']
).configure_view(
    strokeOpacity=0
)
titles_per_country

#Concatenate the other two plots to this one to give a comprehensive visual for all three points of interest.

ValueError: 
To use the 'notebook' renderer, you must install the vega package
and the associated Jupyter extension.
See https://altair-viz.github.io/getting_started/installation.html
for more information.
